### 1. **Фильтрация и сортировка на основе временных диапазонов**

#### **Фильтрация записей по временным интервалам**

Фильтрация по временным диапазонам позволяет получить данные, соответствующие определённому временному промежутку. Например, это может быть:
   - Записи за последние 7 дней.
   - Данные за текущий месяц или год.
   - Прошедшие выходные или конкретный день.

Чтобы фильтровать данные по дате, сначала убедитесь, что в модели используется поле типа `DateField` или `DateTimeField`, например:

In [ ]:
from django.db import models

class Order(models.Model):
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)

##### Пример фильтрации по последним 7 дням:

Для получения записей за последние 7 дней используем функцию `timezone.now()` и `timedelta` для создания временного диапазона.


In [ ]:
from django.utils import timezone
from datetime import timedelta

seven_days_ago = timezone.now() - timedelta(days=7)
recent_orders = Order.objects.filter(created_at__gte=seven_days_ago)

Здесь:
   - `timezone.now()` возвращает текущую дату и время, учитывая часовой пояс.
   - `timedelta(days=7)` создаёт временной интервал в 7 дней.
   - `created_at__gte=seven_days_ago` фильтрует записи, созданные в последние 7 дней.


##### Пример фильтрации по текущему месяцу:

Фильтрация по месяцу требует доступа к началу месяца, что можно сделать через атрибуты даты:

In [ ]:
now = timezone.now()
start_of_month = now.replace(day=1)
orders_this_month = Order.objects.filter(created_at__gte=start_of_month)

Здесь:
   - `now.replace(day=1)` устанавливает день текущего месяца на 1, чтобы получить начало месяца.

##### Фильтрация по текущему году:

In [ ]:
orders_this_year = Order.objects.filter(created_at__year=timezone.now().year)

Здесь:
   - `created_at__year=timezone.now().year` извлекает год из поля `created_at` и сравнивает его с текущим годом.


#### **Использование библиотек и функций Python для работы с временными фильтрами**

Для работы с временными интервалами важно правильно использовать библиотеки `datetime` и `timezone`. Ниже приведены основные функции и приёмы:

   - **`timezone`**: предоставляет текущую дату и время с учётом часового пояса, что важно для корректной работы с данными в разных временных зонах.
   - **`timedelta`**: позволяет устанавливать промежутки времени (например, `timedelta(days=7)` для недели, `timedelta(hours=1)` для часа).

**Пример: получение заказов, созданных за последние 30 дней:**


In [ ]:
thirty_days_ago = timezone.now() - timedelta(days=30)
orders_last_30_days = Order.objects.filter(created_at__gte=thirty_days_ago)

#### **Примеры динамической сортировки по дате**

В Django ORM сортировка выполняется методом `order_by()`, который принимает поле для сортировки. Для временных данных часто используется сортировка по дате создания или обновления.

##### Сортировка по дате создания (новые записи в начале):

In [ ]:
recent_orders = Order.objects.all().order_by('-created_at')

Здесь:
   - `-created_at` означает обратный порядок, т.е. от самых новых к старым.

##### Сортировка по дате обновления (сначала недавно изменённые записи):

In [ ]:
updated_orders = Order.objects.all().order_by('-updated_at')

#### **Совмещение фильтрации и сортировки по дате**

Вы можете комбинировать фильтрацию и сортировку, чтобы получить записи, созданные в определённый временной промежуток, отсортированные по возрастанию или убыванию даты.

**Пример: записи за последний месяц, отсортированные от самых старых к самым новым:**


In [ ]:
start_of_month = timezone.now().replace(day=1)
orders_this_month_sorted = Order.objects.filter(created_at__gte=start_of_month).order_by('created_at')

<span style="color: blue;"><h3>**Совет**</h3></span> Если вам нужно часто фильтровать и сортировать по дате, можно создать менеджер или использовать метод в модели для повторного использования логики:

In [ ]:
class OrderManager(models.Manager):
    def last_seven_days(self):
        seven_days_ago = timezone.now() - timedelta(days=7)
        return self.filter(created_at__gte=seven_days_ago).order_by('-created_at')

class Order(models.Model):
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    objects = OrderManager()

# Использование
recent_orders = Order.objects.last_seven_days()

Такой подход облегчает управление данными и делает код более читаемым и организованным.